In [88]:
#name: Deepak Gupta
#roll no: 167507

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn import datasets
from PIL import Image,ImageDraw,ImageFont

# font =ImageFont.truetype("fontfamily.ttf",size=20)


In [89]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target
print(Y)
print(X.shape)
Attr = (iris.feature_names)
print(Attr)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
(150, 4)
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [90]:
# print(X)
#translate values into range i.e .2 .3 .... will be merged into 1 i.e. all values are in 0 to 1


print(X[0:5,])
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i][j]=(float)((int)(X[i][j]+1))
print(X[0:5,])

print(type(X))
print(type(Y))



[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]]
[[ 6.  4.  2.  1.]
 [ 5.  4.  2.  1.]
 [ 5.  4.  2.  1.]
 [ 5.  4.  2.  1.]
 [ 6.  4.  2.  1.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [91]:
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freq = counts.astype('float')/len(s)
    for i in freq:
        if i != 0.0:
            res -= i*np.log2(i)
    return res


In [92]:
def gain(s,a):
    #res = entropy(s)
    res=0;
    val, counts = np.unique(a, return_counts=True)
    freq = counts.astype('float')/len(a)
    
    for i, j in zip(freq, val):
        entr = entropy(s[a == j])
#         print("Entropy of %s is %f" % (str(j),entr))
#         res -= i*entr
        res += i*entr
    return res

In [93]:
def numberOfClass(s):
    return len(set(s))

In [94]:
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

In [95]:
def split(x,y):
#     print("split ----> start")
    if numberOfClass(y) == 1: # no need to classifiy all values are same.
#         print("---->NO gain identification")
        return y
    
#     gain_attr = np.array([gain(y, x_attr) for x_attr in x.T])
#     print("----------GAINS-----------")
#     print(Attr)
#     print(gain_attr)

    entropy_y = entropy(y)
    entropy_attr = np.array([gain(y, x_attr) for x_attr in np.transpose(x)]) #x.T
    
    gain_attr = np.zeros(len(entropy_attr))
    print("----------Entropy of Attribute-----------")
    print(Attr)
    print(entropy_attr)
    
    for i in range(len(entropy_attr)):
        gain_attr[i]=(entropy_y-entropy_attr[i])
    
    print("----------GAINS-----------")
    print(Attr)
    print(gain_attr)
    
    
    max_gain_attr = np.argmax(gain_attr)
    print("selected attribute having max gain: %s" % Attr[max_gain_attr])
    print()
    
    if np.all(gain_attr < 1e-6):
#         print("---->NO max gain identification")
        return y
    
    partition_set = partition(x[:, max_gain_attr])
    #print(partition_set)
    
    res = {}
    
#     print("split -----> end")
#     print(partition_set.items())
    for i,j in partition_set.items():
        y_subset = y.take(j, axis = 0)
        x_subset = x.take(j, axis = 0)
        
        #print(y_subset)
        #print(x_subset)
        temp = split(x_subset, y_subset)
#         print(temp)
        res["%s = %s" % (Attr[max_gain_attr], i)] = temp
#     res["x_%d = %s" % (max_gain_attr, i)] = temp

    return res
    

In [96]:
def isLeaf(elem):
    if type(elem) is np.ndarray:
        return True
    else:
        return False

In [97]:
def printNode(tree):
#     if istype()
#     print(type(tree))
    for elem in tree:
#         temp = elem
#         temp += ("-->")
#         print(temp)
        print(elem)
#         print(type(tree[elem]))
        if isLeaf(tree[elem]) :
            print(tree[elem][0])
#             temp += (tree[elem][0])
            pass
        else:
            printNode(tree[elem])
#             temp += (printNode(tree[elem]))
#         temp += ("\n")
#     return temp

In [98]:
def getwidth(tree):
    if type(tree) is np.ndarray:
        return 1
    width = 0
    for elem in tree:
        if isLeaf(tree[elem]):
            width += 1
        else:
            width += getwidth(tree[elem])
    return width
        

In [99]:
def getdepth(tree):
    if type(tree) is np.ndarray:
        return 0
    depth = 0
    temp = []
    for elem in tree:
        if isLeaf(tree[elem]):
            depth += 1
        else:
            temp.append(getdepth(tree[elem]))
            depth += np.max(temp)
                
    return depth


In [103]:
def drawtree(tree,jpeg='tree.jpg'):
    w=getwidth(tree)*500
    h=getdepth(tree)*500+150
    print(w)
    print(h)

    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)

    drawnode(draw,tree,w/2,200)
    str1 = [chr(97+3),chr(97+4),chr(97+4),chr(97+15),chr(97),chr(97+10)]
    str1=''.join(str1)
    draw.text((w/2+200,100),str(str1),(0,0,0))
    img.save(jpeg,'JPEG')


In [104]:
def drawnode(draw,tree,x,y):
    if isLeaf(tree):
        draw.text((x-20,y-10),str(tree[0]),(0,0,0))
        return 
    
    w = []
    for elem in tree:
#         print(elem)
        w.append(getwidth(tree[elem])*10)

    print(w)
    avg_dim = np.sum(w)
    x_coord = []
    y_coord = []
    text_offset =20
    text_width = len(elem)/2

    x_coord.append(x-(avg_dim/2))
    y_coord.append(y+200)
    
    for i in range(0,len(w)-1):        
        x_coord.append(x_coord[i]+w[i]*25+(avg_dim/(len(w))) )
        y_coord.append(y+200)
    print(x_coord)
 
    
    for i in range(len(w)):
        draw.line((x,y,x_coord[i], y_coord[i]), fill=(0,0,255),width = 4)
        draw.text((x_coord[i]-50, y_coord[i]),str(list(tree.keys())[i]),(0,0,0))
        draw.line((x_coord[i], y_coord[i]+text_offset,x_coord[i],y_coord[i]+50), fill=(255,0,0), width = 4)
        y_coord[i] += 50+text_offset
        
    # Draw the level 2 and remaining
    c = 0 
    for elem in tree:
        if isLeaf(tree[elem]):
            draw.text((x_coord[c], y_coord[c]),str(tree[elem][0]),(0,0,0))
        else:
            drawnode(draw,tree[elem],x_coord[c],y_coord[c]-text_offset)
        c += 1
        

In [105]:
print("Entropy of training data set %f " % entropy(Y))
tree = split(X,Y)
# print(type(tree))
print(tree)
# print(tree['Outlook = Rain'])

# (printNode(tree))

print(getwidth(tree))
print(getdepth(tree))
drawtree(tree)

Entropy of training data set 1.584963 
----------Entropy of Attribute-----------
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[ 1.01906063  1.31004109  0.2408649   0.41467047]
----------GAINS-----------
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[ 0.56590187  0.27492141  1.3440976   1.17029203]
selected attribute having max gain: petal length (cm)

----------Entropy of Attribute-----------
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[ 0.49428791  0.58110341  0.58301942  0.51437016]
----------GAINS-----------
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[ 0.08873151  0.00191601  0.          0.06864926]
selected attribute having max gain: sepal length (cm)

----------Entropy of Attribute-----------
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[ 0.33729007  0.29312225  0.33729007  0.        ]
------